In [1]:
import os

In [2]:
import json

In [3]:
import numpy as np
import pandas as pd

In [4]:
import tensorflow as tf

In [5]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [6]:
# from sklearn.externals import joblib

/home/developer/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)

In [7]:
import joblib

In [8]:
# a function to encode float values for serialized examples
def _float_feature_from_list_of_values(list_of_values):
    """Returns a float_list from a list of floats / doubles."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=list_of_values))

In [9]:
data_folder = '/home/developer/gcp/cbidmltsf/datasets/electricity'

In [10]:
# constant values for positional encodings
hours_in_day = 24
days_in_week = 7
days_in_month = 30
days_in_year = 365
# weeks_of_year and month_of_year become redundant when using days_of_year, do not evaluate them
# weeks_in_year = 52
# months_in_year = 12

In [11]:
# split the time series in seen (train, eval) and unseen (test) data
# according to academic papers:

# 243 days on seen data, 7 days on unseen data 

# seen data:      '2014-01-01 00:00:00' to '2014-08-31 23:00:00', 243*24 = 5832 lectures

# train/eval split is 0.9/0.1, then

# train data:     '2014-01-01 00:00:00' to '2014-08-07 15:00:00', 5248 lectures
# eval data:      '2014-08-07 15:00:00' to '2014-08-31 23:00:00', 584 lectures

# unseen data:    '2014-09-01 00:00:00' to '2014-09-07 23:00:00', 7*24 = 168 lectures

# 243 weeks for seen data, 1 week for unseen data
no_lectures_seen_data = 243*24 # 5832

# seen data is divided as 90% for training and 10% for evaluation
train_eval_limit = 0.9

train_interval_end = int(no_lectures_seen_data*train_eval_limit) # 5248

In [12]:
# build sub-series to be persisted as serialized training examples

# dimensionality of the encoder input
m = 168

# dimensionality of the decoder output 
t = 168

In [13]:
# columns to be included in the SLDB
sldb_columns = [
    'date',
    'token_id',
    'kw_scaled',
    'sin_hour_day',
    'cos_hour_day',
    'sin_day_week',
    'cos_day_week',
    'sin_day_month',
    'cos_day_month',
    'sin_day_year',
    'cos_day_year'    
]

In [14]:
sldb = {
    'ts': 'LD2011-2014_FULL',
    'embedding': {
        'hourly': 168
    },
    'tau': {
        'hourly': 1
    },
    'no_targets': 168,
    'BSCTRFM': 1
}

In [15]:
sldb

{'ts': 'LD2011-2014_FULL',
 'embedding': {'hourly': 168},
 'tau': {'hourly': 1},
 'no_targets': 168,
 'BSCTRFM': 1}

In [16]:
# a string with the basic specifications of the SLDB, as part of the SLDB identifier
sldb_specs = 'BSCTRFM_{:03d}_{:03d}'.format(sldb['embedding']['hourly'], sldb['no_targets'])
sldb_specs

'BSCTRFM_168_168'

In [17]:
# get the time-based identifier for the SLDB
sldb_identifier = '{}_{}'.format(sldb['ts'], sldb_specs)
sldb_identifier

'LD2011-2014_FULL_BSCTRFM_168_168'

In [18]:
sldb_dir = '/home/developer/gcp/cbidmltsf/sldbs/{}'.format(sldb_identifier)
sldb_dir

'/home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_FULL_BSCTRFM_168_168'

In [19]:
# make a directory for the complete SLDB
try:
    os.mkdir(sldb_dir)
    print('Directory {} was created.'.format(sldb_dir))
except FileExistsError:
    print('Error: directory {} already exists.'.format(sldb_dir))

Error: directory /home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_FULL_BSCTRFM_168_168 already exists.


In [20]:
# make a sub-directory for the training TFRecord files
try:
    os.mkdir('{}/train'.format(sldb_dir))
    print('Directory {}/train was created.'.format(sldb_dir))
except FileExistsError:
    print('Error: directory {}/train already exists.'.format(sldb_dir))

Error: directory /home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_FULL_BSCTRFM_168_168/train already exists.


In [21]:
# make a sub-directory for the evaluation TFRecord files
try:
    os.mkdir('{}/eval'.format(sldb_dir))
    print('Directory {}/eval was created.'.format(sldb_dir))
except FileExistsError:
    print('Error: directory {}/eval already exists.'.format(sldb_dir))

Error: directory /home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_FULL_BSCTRFM_168_168/eval already exists.


In [22]:
# get a path to the scalers sub-directory
scalers_dir = '{}/scalers'.format(sldb_dir)
scalers_dir

'/home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_FULL_BSCTRFM_168_168/scalers'

In [23]:
# make a sub-directory for the scalers
try:
    os.mkdir(scalers_dir)
    print('Directory {} was created.'.format(scalers_dir))
except FileExistsError:
    print('Error: directory {} already exists.'.format(scalers_dir))

Error: directory /home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_FULL_BSCTRFM_168_168/scalers already exists.


In [24]:
encoder_input_columns = [
    'kw_scaled',
    'sin_hour_day',
    'cos_hour_day',
    'sin_day_week',
    'cos_day_week',
    'sin_day_month',
    'cos_day_month',
    'sin_day_year',
    'cos_day_year'
]

In [25]:
# both the encoder input and the decoder input use the same columns from the source sub_series dataframe
decoder_input_columns = encoder_input_columns

In [26]:
target_columns = ['kw_scaled']

In [27]:
id_columns = ['token_id']

In [28]:
# do not read the complete dataset!

# output = pd.read_pickle('{}/hourly_electricity_complete.pkl'.format(data_folder))

In [29]:
# do not filter the complete dataset!

# filter to match range used by other academic papers
# filtered_output = output[(output['days_from_start'] >= 1096) & (output['days_from_start'] < 1346)].copy()

In [30]:
# do not persist the filtered dataset again!

# persist the filtered dataset to avoid using memory for the complete dataset
# filtered_output.to_pickle('{}/hourly_electricity_filtered_academic_papers.pkl'.format(data_folder))

In [31]:
# just read the filtered_output dataframe
filtered_output = pd.read_pickle('{}/hourly_electricity_filtered_academic_papers.pkl'.format(data_folder))

In [46]:
# a dictionary to manage data per individual customer_id
data = dict()

In [47]:
# a dictionary to manage a MinMaxScaler per individual customer_id
min_max = dict()
# a dictionary to manage a StandardScaler per individual customer_id
standard = dict()

In [48]:
# the number of cores available for training in Cloud TPU
num_cores = 8

In [49]:
start, end = 301, 370

In [50]:
token_ids = [token_id for token_id in np.arange(start, end + 1)]

In [51]:
customer_ids = ['MT_{:03d}'.format(token_id) for token_id in token_ids]

### for SLDB generation, run this unified code!

In [52]:
for token_id in token_ids:
    
    # initialize the examples dictionary for each customer
    examples = {
        'train': [],
        'eval': [],
        # test dataset is not passed to SLDB
        # 'test': []
    }

    # get the customer identifier
    customer_id = 'MT_{:03d}'.format(token_id)
    customer_id
    print('Started processing for {}'.format(customer_id))

    # a temporary dataframe with data per customer_id to build the sub-series/examples
    data_df = filtered_output[filtered_output['token_id'] == token_id].copy()

    # expand with positional encodings
    data_df['sin_hour_day'] = np.sin(2*np.pi*data_df.hour_of_day/hours_in_day)
    data_df['cos_hour_day'] = np.cos(2*np.pi*data_df.hour_of_day/hours_in_day)
    data_df['sin_day_week'] = np.sin(2*np.pi*data_df.day_of_week/days_in_week)
    data_df['cos_day_week'] = np.cos(2*np.pi*data_df.day_of_week/days_in_week)
    data_df['sin_day_month'] = np.sin(2*np.pi*data_df.day_of_month/days_in_month)
    data_df['cos_day_month'] = np.cos(2*np.pi*data_df.day_of_month/days_in_month)
    data_df['sin_day_year'] = np.sin(2*np.pi*data_df.day_of_year/days_in_year)
    data_df['cos_day_year'] = np.cos(2*np.pi*data_df.day_of_year/days_in_year)

    # get a series for the power usage variable on the training dataset, to fit the scaler
    lectures_train_data = data_df['power_usage'][:train_interval_end]

    # fit a scaler only on train data
    # it is required to pass the power usage time series to a (?, 1) NumPy array
    lectures_train_data_array = np.array(lectures_train_data).reshape(-1, 1)

    # get MinMaxScaler on train data, store it in a dictionary
    min_max_scaler = MinMaxScaler()
    min_max = min_max_scaler.fit(lectures_train_data_array)
    print('MinMax scaler generated on training data for {}'.format(customer_id))

    # persist the scaler
    scaler_filename = '{}/{}_min_max.save'.format(scalers_dir, customer_id)
    joblib.dump(min_max, scaler_filename)
    print('MinMax scaler persisted for {}'.format(customer_id))

    
    # get an array from the variable time series (seen and unseen)
    all_data_variable_array = np.array(data_df.power_usage).reshape(-1, 1)

    # apply the scaler over all data (seen and unseen)
    # rescale, and squeeze to drop the extra dimension, then assign to the new column kw_scaled
    data_df['kw_scaled'] = np.squeeze(min_max.transform(all_data_variable_array))

    # get an iterable with all the possible sub-series for training examples
    for starting_point in np.arange(train_interval_end - (m + t) + 1):

        sub_series_df = data_df[sldb_columns][starting_point:starting_point + (m + t)]

        encoder_input_df = sub_series_df[encoder_input_columns][:m]
        decoder_input_df = sub_series_df[decoder_input_columns][m-1:m-1+t]
        target_df = sub_series_df[target_columns][m:m+t]
        id_df = sub_series_df[id_columns][:1]

        encoder_input_list = encoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
        decoder_input_list = decoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
        target_list = target_df.reset_index(drop=True).to_numpy().flatten().tolist()
        id_list = id_df.reset_index(drop=True).to_numpy().flatten().tolist()

        examples['train'].append(
            {
                'encoder_input': encoder_input_list,
                'decoder_input': decoder_input_list,
                'target': target_list,
                'id': id_list,
            }
        )
    
    print('{} processed. The number of examples in {} dataset is {}'.\
          format(customer_id, 'train', len(examples['train'])))
    
    
    # ToDo: remove evaluation step from Cloud TPU training and use all seen data for training stage
    build_eval_set = True
    
    if build_eval_set:

        # get an iterable with all the possible sub-series for evaluation examples
        for starting_point in np.arange(train_interval_end, no_lectures_seen_data - (m + t) + 1):

            sub_series_df = data_df[sldb_columns][starting_point:starting_point + (m + t)]

            encoder_input_df = sub_series_df[encoder_input_columns][:m]
            decoder_input_df = sub_series_df[decoder_input_columns][m-1:m-1+t]
            target_df = sub_series_df[target_columns][m:m+t]
            id_df = sub_series_df[id_columns][:1]

            encoder_input_list = encoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
            decoder_input_list = decoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
            target_list = target_df.reset_index(drop=True).to_numpy().flatten().tolist()
            id_list = id_df.reset_index(drop=True).to_numpy().flatten().tolist()

            examples['eval'].append(
                {
                    'encoder_input': encoder_input_list,
                    'decoder_input': decoder_input_list,
                    'target': target_list,
                    'id': id_list,
                }
            )

        print('{} processed. The number of examples in {} dataset is {}'.\
              format(customer_id, 'eval', len(examples['eval'])))
    
    
    # DO NOT PRODUCE A TEST DATASET FOR SLDB, AS INFERENCE PROCESS IS NOT DIRECT
    # (IT IS ITERATIVE OVER UNSEEN DATA TIME SERIES)
    
    build_test_set = False
    
    if build_test_set:

        # remember that conditional range of test dataset overlaps with evaluation dataset
        # for this experiment design# get an iterable with all the possible sub-series for test examples
        for starting_point in no_lectures_seen_data - (m + t) + 1 + np.arange(168):

            sub_series_df = data_df[sldb_columns][starting_point:starting_point + (m + t)]

            encoder_input_df = sub_series_df[encoder_input_columns][:m]
            decoder_input_df = sub_series_df[decoder_input_columns][m-1:m-1+t]
            target_df = sub_series_df[target_columns][m:m+t]
            id_df = sub_series_df[id_columns][:1]

            encoder_input_list = encoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
            decoder_input_list = decoder_input_df.reset_index(drop=True).to_numpy().flatten().tolist()
            target_list = target_df.reset_index(drop=True).to_numpy().flatten().tolist()
            id_list = id_df.reset_index(drop=True).to_numpy().flatten().tolist()

            examples['test'].append(
                {
                    'encoder_input': encoder_input_list,
                    'decoder_input': decoder_input_list,
                    'target': target_list,
                    'id': id_list,
                }
            )

        print('{} processed. The number of examples in {} dataset is {}'.\
              format(customer_id, 'test', len(examples['test'])))
    
    
    # on each customer dataset, adjust the number of examples to the number of training cores
    for stage in ['train', 'eval']:
        # how many examples/rows must be removed from examples[stage] to comply with the number of cores
        examples_to_remove = len(examples[stage])%num_cores

        # remove the last 'examples_to_remove' examples from the dataset
        for _ in np.arange(examples_to_remove):
            examples[stage].pop(-1)

        print('For {} cores in Cloud TPU, the number of {} examples for {} was adjusted to {}'.\
             format(num_cores, stage, customer_id, len(examples[stage])))


    # serialize the rows in examples['train'] and, if present, examples['eval']
    # to avoid excesive memory consumption
    
    # write a TFRecord file for each consumer_id/stage
    for stage in ['train', 'eval']:
        # N_ROWS = sldb['stats'][stage]['n_rows']
        N_ROWS = len(examples[stage])
        filename = '{}/{}/{}.tfrecord'.format(sldb_dir, stage, customer_id)

        with tf.io.TFRecordWriter(filename) as writer:
            for row in np.arange(N_ROWS):

                example = tf.train.Example(
                    # features within the example
                    features=tf.train.Features(
                        # individual feature definition
                        feature={'encoder_input':
                                 _float_feature_from_list_of_values(
                                     examples[stage][row]['encoder_input']),
                                 'decoder_input':
                                 _float_feature_from_list_of_values(
                                     examples[stage][row]['decoder_input']),
                                 'target':
                                 _float_feature_from_list_of_values(
                                     examples[stage][row]['target']),
                                 'id':
                                 _float_feature_from_list_of_values(
                                     examples[stage][row]['id'])
                                 }
                    )
                )
                serialized_example = example.SerializeToString()
                writer.write(serialized_example)
                
            # report TFRecord file as completed
            print('Persisted {} TFRecord file for {}'.format(stage, customer_id))

Started processing for MT_301
MinMax scaler generated on training data for MT_301
MinMax scaler persisted for MT_301
MT_301 processed. The number of examples in train dataset is 4913
MT_301 processed. The number of examples in eval dataset is 249
For 8 cores in Cloud TPU, the number of train examples for MT_301 was adjusted to 4912
For 8 cores in Cloud TPU, the number of eval examples for MT_301 was adjusted to 248
Persisted train TFRecord file for MT_301
Persisted eval TFRecord file for MT_301
Started processing for MT_302
MinMax scaler generated on training data for MT_302
MinMax scaler persisted for MT_302
MT_302 processed. The number of examples in train dataset is 4913
MT_302 processed. The number of examples in eval dataset is 249
For 8 cores in Cloud TPU, the number of train examples for MT_302 was adjusted to 4912
For 8 cores in Cloud TPU, the number of eval examples for MT_302 was adjusted to 248
Persisted train TFRecord file for MT_302
Persisted eval TFRecord file for MT_302


Persisted train TFRecord file for MT_317
Persisted eval TFRecord file for MT_317
Started processing for MT_318
MinMax scaler generated on training data for MT_318
MinMax scaler persisted for MT_318
MT_318 processed. The number of examples in train dataset is 4913
MT_318 processed. The number of examples in eval dataset is 249
For 8 cores in Cloud TPU, the number of train examples for MT_318 was adjusted to 4912
For 8 cores in Cloud TPU, the number of eval examples for MT_318 was adjusted to 248
Persisted train TFRecord file for MT_318
Persisted eval TFRecord file for MT_318
Started processing for MT_319
MinMax scaler generated on training data for MT_319
MinMax scaler persisted for MT_319
MT_319 processed. The number of examples in train dataset is 4913
MT_319 processed. The number of examples in eval dataset is 249
For 8 cores in Cloud TPU, the number of train examples for MT_319 was adjusted to 4912
For 8 cores in Cloud TPU, the number of eval examples for MT_319 was adjusted to 248


MT_334 processed. The number of examples in train dataset is 4913
MT_334 processed. The number of examples in eval dataset is 249
For 8 cores in Cloud TPU, the number of train examples for MT_334 was adjusted to 4912
For 8 cores in Cloud TPU, the number of eval examples for MT_334 was adjusted to 248
Persisted train TFRecord file for MT_334
Persisted eval TFRecord file for MT_334
Started processing for MT_335
MinMax scaler generated on training data for MT_335
MinMax scaler persisted for MT_335
MT_335 processed. The number of examples in train dataset is 4913
MT_335 processed. The number of examples in eval dataset is 249
For 8 cores in Cloud TPU, the number of train examples for MT_335 was adjusted to 4912
For 8 cores in Cloud TPU, the number of eval examples for MT_335 was adjusted to 248
Persisted train TFRecord file for MT_335
Persisted eval TFRecord file for MT_335
Started processing for MT_336
MinMax scaler generated on training data for MT_336
MinMax scaler persisted for MT_336


Persisted train TFRecord file for MT_350
Persisted eval TFRecord file for MT_350
Started processing for MT_351
MinMax scaler generated on training data for MT_351
MinMax scaler persisted for MT_351
MT_351 processed. The number of examples in train dataset is 4913
MT_351 processed. The number of examples in eval dataset is 249
For 8 cores in Cloud TPU, the number of train examples for MT_351 was adjusted to 4912
For 8 cores in Cloud TPU, the number of eval examples for MT_351 was adjusted to 248
Persisted train TFRecord file for MT_351
Persisted eval TFRecord file for MT_351
Started processing for MT_352
MinMax scaler generated on training data for MT_352
MinMax scaler persisted for MT_352
MT_352 processed. The number of examples in train dataset is 4913
MT_352 processed. The number of examples in eval dataset is 249
For 8 cores in Cloud TPU, the number of train examples for MT_352 was adjusted to 4912
For 8 cores in Cloud TPU, the number of eval examples for MT_352 was adjusted to 248


MT_367 processed. The number of examples in train dataset is 4913
MT_367 processed. The number of examples in eval dataset is 249
For 8 cores in Cloud TPU, the number of train examples for MT_367 was adjusted to 4912
For 8 cores in Cloud TPU, the number of eval examples for MT_367 was adjusted to 248
Persisted train TFRecord file for MT_367
Persisted eval TFRecord file for MT_367
Started processing for MT_368
MinMax scaler generated on training data for MT_368
MinMax scaler persisted for MT_368
MT_368 processed. The number of examples in train dataset is 4913
MT_368 processed. The number of examples in eval dataset is 249
For 8 cores in Cloud TPU, the number of train examples for MT_368 was adjusted to 4912
For 8 cores in Cloud TPU, the number of eval examples for MT_368 was adjusted to 248
Persisted train TFRecord file for MT_368
Persisted eval TFRecord file for MT_368
Started processing for MT_369
MinMax scaler generated on training data for MT_369
MinMax scaler persisted for MT_369


In [63]:
# expand the sldb dictionary with final statistics

In [66]:
sldb['stats'] = {
    'train': {
        'n_rows': len(examples['train'])
    },
    'eval': {
        'n_rows': len(examples['eval'])
    }
}

In [67]:
sldb

{'ts': 'LD2011-2014_FULL',
 'embedding': {'hourly': 168},
 'tau': {'hourly': 1},
 'no_targets': 168,
 'BSCTRFM': 1,
 'stats': {'train': {'n_rows': 4912}, 'eval': {'n_rows': 248}}}

In [64]:
# serialize the sldb from the examples dictionary (keys are stages, values are lists of rows)

In [101]:
json_filename = '{}/sldb.json'.format(sldb_dir)

In [102]:
with open(json_filename, 'w') as filename:
    json.dump(sldb, filename, indent=4)

### ToDo: read TFRecord file into a Dataset and confirm the values in the source dataframe!!!

In [68]:
# do not forget to sync sldbs/ from local to GS after the previous operations!
# !gsutil rsync -d -r /home/developer/gcp/cbidmltsf/sldbs gs://cbidmltsf/sldbs

Building synchronization state...
Starting synchronization...
Copying file:///home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_MT320-MT321_BSCTRFM_168_168/eval.tfrecord [Content-Type=application/octet-stream]...
Copying file:///home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_MT320-MT321_BSCTRFM_168_168/sldb.json [Content-Type=application/json]...
Copying file:///home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_MT320-MT321_BSCTRFM_168_168/test.tfrecord [Content-Type=application/octet-stream]...
Copying file:///home/developer/gcp/cbidmltsf/sldbs/LD2011-2014_MT320-MT321_BSCTRFM_168_168/train.tfrecord [Content-Type=application/octet-stream]...
- [4 files][130.9 MiB/130.9 MiB]    4.2 MiB/s                                   
Operation completed over 4 objects/130.9 MiB.                                    


In [ ]:
# ToDo: random sample the serialization of examples to TFRecord SLDB!!!